# Databricks RAGアプリチュートリアル - 参考: 日本語埋め込みモデルのデプロイ

このノートブックでは、日本語対応の埋め込みモデル[pfnet/plamo-embedding-1b](https://huggingface.co/pfnet/plamo-embedding-1b)をDatabricks上でMLflow登録からモデルサービングエンドポイントの作成まで、一連のデプロイフローを実行します。

## このノートブックで学習する内容
1. **Hugging Face モデルの取得**とローカルでの動作確認
2. **MLflowモデル登録**：`sentence_transformers`フレーバーでの登録方法
3. **Unity Catalogモデル管理**：企業レベルのモデルガバナンス
4. **モデルサービングエンドポイント作成**：本番利用可能なAPI提供
5. **エンドポイントテスト**：デプロイしたモデルの動作確認

## 前提条件

ワークショップ共通の動作条件はREADME.mdをご確認ください。

- **動作確認済の環境**：16.3 ML Runtime
- **推奨インスタンス**：AWS の `i3.xlarge` または Azure の `Standard_D4DS_v5`  
- **Unity Catalog**：ワークスペースでUnity Catalogが有効化されていること
- **GPU Model Serving**：GPUモデルサービングへのアクセス権限があること
- **必要な権限**：
  - Unity Catalogでのカタログ・スキーマ作成権限
  - モデル登録・サービングエンドポイント作成権限

--- 


In [0]:
# カタログ、スキーマ、モデル名の定義
# TODO: 
model_uc_catalog = "skato"
model_uc_schema = "models"
uc_model_name = "plamo_embedding_1b"

# サービングエンドポイントの名前を指定
serving_endpoint_name = 'skato-plamo-embedding'

## 1. パラメータの設定

日本語埋め込みモデルのデプロイに必要なパラメータを設定します。

### Unity Catalogの設定について
- **カタログ**: Unity Catalogの最上位レベルのコンテナ
- **スキーマ**: カタログ内のモデル管理用スキーマ  
- **モデル名**: Unity Catalogでの登録モデル名

### サービングエンドポイントについて
GPU Model Servingを使用して、リアルタイムでの埋め込み生成APIを提供します。

In [0]:
import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

# You can download models from the Hugging Face Hub 🤗 as follows:
tokenizer = AutoTokenizer.from_pretrained("pfnet/plamo-embedding-1b", trust_remote_code=True)
model = AutoModel.from_pretrained("pfnet/plamo-embedding-1b", trust_remote_code=True)

## 2. モデルの取得とローカルテスト

### Hugging Face Hubからのモデル取得

PLaMo-Embedding-1Bは、Preferred Networks社が開発した日本語特化の埋め込みモデルです。Hugging Face Transformersライブラリを使用してモデルとトークナイザーを取得し、ローカルでの動作を確認します。

### モデルの特徴
- **言語特化**: 日本語テキストに最適化された埋め込み生成
- **用途別メソッド**: `encode_query`（クエリ用）と`encode_document`（文書用）の使い分け
- **高精度**: 日本語の情報検索タスクで高い性能を発揮

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

query = "PLaMo-Embedding-1Bとは何ですか？"
documents = [
    "PLaMo-Embedding-1Bは、Preferred Networks, Inc. によって開発された日本語テキスト埋め込みモデルです。",
    "最近は随分と暖かくなりましたね。"
]

with torch.inference_mode():
    # クエリテキストの埋め込み生成には `encode_query` メソッドを使用します。
    # `tokenizer` も一緒に渡す必要があります。
    query_embedding = model.encode_query(query, tokenizer)
    # 他のテキストや文の場合は `encode_document` メソッドを使用してください。
    # また、情報検索以外の用途でも `encode_document` メソッドを使用してください。
    document_embeddings = model.encode_document(documents, tokenizer)


similarities = F.cosine_similarity(query_embedding, document_embeddings)
print(similarities)

## 3. モデルのMLflow登録

### MLflow Transformersフレーバーについて

MLflowのTransformersフレーバーを使用して、Hugging Faceモデルを企業レベルのモデル管理システムに統合します。

**登録のメリット:**
- **バージョン管理**: モデルの変更履歴とメタデータの追跡
- **再現性**: モデルの再利用と一貫性の確保  
- **ガバナンス**: Unity Catalogによるアクセス制御と監査
- **デプロイメント**: サービングエンドポイントへの直接デプロイ


In [0]:
import mlflow
from transformers import pipeline
import numpy as np

transformers_model = {"model": model, "tokenizer": tokenizer}
task = "llm/v1/embeddings"  # 埋め込み用の正しいタスク

# レジストリURIをUnity Catalogに設定
mlflow.set_registry_uri("databricks-uc")

# パイプラインを作成
embedding_pipeline = pipeline(task="feature-extraction", model=model, tokenizer=tokenizer)

# モデルを登録
with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=embedding_pipeline,
        task=task,
        artifact_path="model",
        registered_model_name=f"{model_uc_catalog}.{model_uc_schema}.{uc_model_name}"
    )


## 4. Unity Catalogからのモデル読み込み

### モデルエイリアス機能

Unity Catalogでは、モデルバージョンにエイリアス（別名）を設定できます。これにより、常に最新バージョンや特定の環境用バージョンを参照できます。

**エイリアスの利点:**
- **バージョン抽象化**: 具体的なバージョン番号を意識せずにモデル参照
- **環境管理**: development、staging、productionなどの環境別管理
- **自動更新**: 新しいバージョンのデプロイ時の自動切り替え

### MLflow Clientによる管理
MLflow Clientを使用して、プログラムでモデルバージョンとエイリアスを管理できます。

In [0]:
import mlflow
from mlflow import MlflowClient

# モデル名とエイリアスを定義
registered_name = f"{model_uc_catalog}.{model_uc_schema}.{uc_model_name}"
alias = "latest_alias"

# 最新バージョンにエイリアスを設定
client = MlflowClient()
latest_version = max([int(m.version) for m in client.search_model_versions(f"name='{registered_name}'")])
client.set_registered_model_alias(registered_name, alias, latest_version)

# エイリアスを使用してモデルを読み込み
loaded_model = mlflow.pyfunc.load_model(f"models:/{registered_name}@{alias}")

## 5. モデルサービングエンドポイントの作成

### Model Servingについて

Model Servingは、GPUアクセラレーションを活用した高性能なモデル推論サービスです。埋め込みモデルのような計算集約的なタスクに最適です。

**主な特徴:**
- **高速推論**: GPUによる並列処理で高速な埋め込み生成
- **自動スケーリング**: トラフィックに応じた動的リソース調整
- **Zero-downtime**: リクエストがない時間帯の自動スケールダウン
- **API統合**: 標準的なREST APIでの簡単なアクセス

### Databricks SDKによるプログラム制御

Databricks SDKを使用することで、UIを使わずにプログラムでエンドポイントの作成・管理が可能です。

In [0]:
databricks_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

In [0]:
# サービングエンドポイントの作成または更新
from datetime import timedelta
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedModelInput, ServedModelInputWorkloadSize, ServedModelInputWorkloadType
import mlflow
from mlflow import MlflowClient

# エイリアスを使用してモデルを読み込み
model_uri = f"models:/{registered_name}@{alias}"
model_name = registered_name

# MLflowクライアントで最新モデルバージョンを取得する必要はありません
# latest_model_version = client.get_latest_versions(model_name, stages=["None"])[0].version

w = WorkspaceClient()
endpoint_config = EndpointCoreConfigInput(
    name=serving_endpoint_name,
    served_models=[
        ServedModelInput(
            model_name=model_name,
            model_version=latest_version,  # Use the version directly
            workload_type=ServedModelInputWorkloadType.GPU_SMALL,
            workload_size=ServedModelInputWorkloadSize.SMALL,
            scale_to_zero_enabled=True
        )
    ]
)

existing_endpoint = next(
    (e for e in w.serving_endpoints.list() if e.name == serving_endpoint_name), None
)
serving_endpoint_url = f"{databricks_url}/ml/endpoints/{serving_endpoint_name}"
if existing_endpoint is None:
    print(f"Creating the endpoint {serving_endpoint_url}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.create_and_wait(name=serving_endpoint_name, config=endpoint_config, timeout=timedelta(minutes=60))
else:
    print(f"Updating the endpoint {serving_endpoint_url} to version {version}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.update_config_and_wait(served_models=endpoint_config.served_models, name=serving_endpoint_name, timeout=timedelta(minutes=60))
    
displayHTML(f'Your Model Endpoint Serving is now available. Open the <a href="/ml/endpoints/{serving_endpoint_name}">Model Serving Endpoint page</a> for more details.')

### エンドポイントの準備確認

モデルサービングエンドポイントのデプロイが完了し、`Ready`状態になったら、以下のテストを実行します。

## 6. サービングエンドポイントのテスト

### MLflow Deployments SDKによるクエリ

モデルサービングエンドポイントが準備完了したら、MLflow Deployments SDKを使用して簡単にテストできます。

**テストのポイント:**
- **入力形式**: 埋め込み生成したいテキストの配列
- **出力形式**: 数値ベクトルとしての埋め込み表現
- **パフォーマンス**: レスポンス時間と精度の確認

### 本番利用への準備
このテストが成功すれば、エンドポイントは本番のRAGアプリケーションで利用可能です。


In [0]:
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")

embeddings_response = client.predict(
    endpoint=serving_endpoint_name,
    inputs={
        "inputs": ["おはようございます"]
    }
)
embeddings_response['predictions']